In [12]:
import pandas as pd 
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
import plotly.express as px
from functions.common_functions import *

In [13]:
df = pd.read_csv('../data/processed/mannich_reaction_descriptors.csv')

In [14]:
## Initial Processing ##

vt = VarianceThreshold()

parameters = df.iloc[:,4:] ## Set parameters to desired columns ##
thres = vt.fit(parameters)
parameters = thres.transform(parameters)

scaled_data = StandardScaler().fit_transform(parameters)

In [15]:
## Elbow Plot ##

num_reactions = len(df)

reducer = umap.UMAP(
    random_state=25,
    min_dist=0,  # Default is 0.1
    n_neighbors=num_reactions-1,  # Default is 15
    n_components=10)

clustering_embedding = reducer.fit_transform(scaled_data)

range_, inertia = get_clusters(clustering_embedding,50)

fig = px.scatter(x=range_, y=inertia,
                labels={'x':'# of Clusters', 'y':'Distortion'},
                template='simple_white'
                )

fig.update_traces(marker=dict(size=17,
                              line=dict(width=2,
                                        color='Black')),
                  selector=dict(mode='markers'))

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

fig.update_layout(
    font_size=20,
)

fig.show()

In [16]:
## Get Cluster Labels ##

labels = k_cluster(clustering_embedding,11)
rxn_cluster_df = pd.DataFrame({'Reaction':df['Reaction'],'cluster_label':labels})


In [17]:
## Merge labels to full dataset ##

full_df = pd.read_csv('../data/processed/mannich_database_processed.csv')
full_df_trim = full_df.loc[:,['Reaction','Catalyst_Type','ee']]
full_df_clusters = pd.merge(full_df_trim,rxn_cluster_df,how='inner',on='Reaction')

In [18]:
## Calculate Generality ##
cats = full_df_clusters['Catalyst_Type'].unique()

cat_list = []
gen_list = []
cluster_list = []
cluster_ee_list = []
for cat in cats:
    generality, clusters, cluster_ee = calc_generality(full_df_clusters,cat,'Catalyst_Type','ee', full_df_clusters['cluster_label'], 70)
    cat_list.append(cat)
    gen_list.append(generality)
    cluster_list.append(clusters)
    cluster_ee_list.append(cluster_ee)

In [19]:
## Plots ##

fig = px.bar(x=cat_list, y=gen_list,
                labels={'x':'Catalyst Class', 'y':'Generality'},
                template='simple_white',
                width=980, height=720,
                )

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

fig.update_layout(
    font_size=20,
)

fig.show()

In [20]:
# Align clusters for plotting
thio = pd.DataFrame({'Cluster':cluster_list[5],'Thiourea':cluster_ee_list[5]})
sec = pd.DataFrame({'Cluster':cluster_list[0],'Secondary Amine':cluster_ee_list[0]})
clusters = pd.DataFrame({'Cluster':np.unique(labels)})
join_df = pd.merge(pd.merge(clusters,thio,how='left', on='Cluster'),sec,how='left',on='Cluster')
join_df = join_df.sort_values('Secondary Amine')
join_df['Cluster'] = range(0,11)

fig = px.bar(join_df, x='Cluster',y=['Thiourea','Secondary Amine'],
                labels={'x':'Cluster', 'y':'Average ee', 'value':'Average ee'},
                template='simple_white',
                barmode='group',
                width=980, height=720,
                color_discrete_sequence=[px.colors.qualitative.T10[2],px.colors.qualitative.T10[0]]
                )


fig.update_traces(marker=dict(size=17,
                              line=dict(width=2,
                                        color='Black')),
                  selector=dict(mode='markers'))

fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

fig.update_layout(
    font_size=20,
)

fig.update_layout(
    legend=dict(
        yanchor="bottom",
        y=1.01,
        xanchor="left",
        x=0.01,
        orientation='h',
    ),
    legend_title_text=''
)

fig.update_xaxes(dtick=1)

fig.update_yaxes(range=[50,100])

fig.show()

In [21]:
full_df = pd.read_csv('../data/processed/mannich_database_processed.csv')
full_df = full_df[full_df['rxn_type']=='Optimization']
ref_df = pd.DataFrame(full_df['Reference'].value_counts())


In [22]:

import plotly.figure_factory as ff

hist_data = [ref_df.Reference]
hist_labels = ['Reference']

colors = [px.colors.qualitative.T10[0],px.colors.qualitative.T10[2]]
fig = ff.create_distplot(hist_data, hist_labels, colors=colors, show_rug=False)

fig.update_layout(
    font_size=20,
    template = 'simple_white',
    width = 980, height=720
)

fig.update_layout(
    legend=dict(
        yanchor="bottom",
        y=0.99,
        xanchor="left",
        x=0.01,
        # bordercolor = 'Black',
        # borderwidth = 1,
        orientation='h',
    ),
    legend_title_text=''
)

fig.update_layout(yaxis_range=[0,0.12], showlegend=False)

fig.update_xaxes(title_text='# of Optimization Reactions in a Manuscript')
fig.update_yaxes(title_text='Density')

# fig.update_layout(showlegend=False)

fig.show()

#fig.write_image('./test.png', scale=2)